<a href="https://colab.research.google.com/github/polugariteja/531-Training-3-2/blob/main/Training_29_12_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
sentences = [
    "this movie is good",
    "i like this film",
    "this movie is bad",
    "i hate this film"
]

labels = [1, 1, 0, 0]
vocab = {}
for sentence in sentences:
    for word in sentence.split():
        if word not in vocab:

            vocab[word] = len(vocab) + 1
vocab_size = len(vocab) + 1
def encode(sentence):
    return [vocab[word] for word in sentence.split()]
encoded_sentences = [encode(s) for s in sentences]



X = torch.tensor(encoded_sentences, dtype=torch.long)
y = torch.tensor(labels, dtype=torch.long)
class TextRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(TextRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)              # (batch, seq_len, embed_size)
        out, _ = self.rnn(x)               # (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])       # last time step
        return out
model = TextRNN(
    vocab_size=vocab_size,
    embed_size=10,
    hidden_size=16,
    output_size=2
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 100

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)             # Forward pass
    loss = criterion(outputs, y)   # Loss calculation

    loss.backward()                # Backpropagation
    optimizer.step()               # Update weights

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
def predict(sentence):
    encoded = torch.tensor([encode(sentence)], dtype=torch.long)
    output = model(encoded)
    prediction = torch.argmax(output, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"

print("\nPredictions:")
print("this movie is good ->", predict("this movie is good"))
print("this movie is bad  ->", predict("this movie is bad"))
print("i like this film  ->", predict("i like this film"))
print("i hate this film  ->", predict("i hate this film"))

Epoch [20/100], Loss: 0.0155
Epoch [40/100], Loss: 0.0016
Epoch [60/100], Loss: 0.0008
Epoch [80/100], Loss: 0.0006
Epoch [100/100], Loss: 0.0005

Predictions:
this movie is good -> Positive
this movie is bad  -> Negative
i like this film  -> Positive
i hate this film  -> Negative
